<a href="https://colab.research.google.com/github/pilar260/DMA_LAB2/blob/master/tutoriales/08_Ejemplo_RNN_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import plotly.graph_objects as go
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [2]:
# Leer un Ticket, frecuencia de la observación = 3 Minutos
intrad = pd.read_csv('https://github.com/Argentan/DMA_LAB2/blob/master/data/AAPL.csv.gz?raw=true', compression='gzip', parse_dates=[0], index_col=[0])

In [3]:
intrad.head()

,Open,High,Low,Close,Volume,WAP,Count,ticket
fecha,,,,,,,,
2020-02-10 11:30:00,314.18,317.36,313.85,317.06,14422,315.243,4194,AAPL
2020-02-10 11:33:00,316.99,317.76,316.84,317.10,5167,317.323,2502,AAPL
2020-02-10 11:36:00,317.15,318.13,316.91,317.95,3712,317.411,1837,AAPL
2020-02-10 11:39:00,317.97,318.95,317.55,318.70,5171,318.253,2613,AAPL
2020-02-10 11:42:00,318.67,318.90,317.94,318.15,2982,318.351,1647,AAPL


In [5]:
intrad.tail()

,Open,High,Low,Close,Volume,WAP,Count,ticket
fecha,,,,,,,,
2009-12-28 17:45:00,30.17,30.21,30.17,30.18,13300,30.191143,1203,AAPL
2009-12-28 17:48:00,30.18,30.21,30.16,30.17,12971,30.181571,967,AAPL
2009-12-28 17:51:00,30.17,30.19,30.16,30.19,10010,30.179286,780,AAPL
2009-12-28 17:54:00,30.18,30.25,30.18,30.23,24906,30.225714,1937,AAPL
2009-12-28 17:57:00,30.24,30.26,30.22,30.23,28868,30.244714,2277,AAPL


In [4]:
intrad.shape

(337010, 8)

In [9]:
# Define Función para Agrupar datos por Hora (H), Dia (D), Semana (W) - Ver al final más opciones

def agrupar_temporalmente(dataset, frecuencia = 'D'):
  df = pd.DataFrame()
  df['Open']   = intrad.Open.resample(frecuencia).first().dropna()
  df['High']   = intrad.High.resample(frecuencia).max().dropna()
  df['Low']    = intrad.Low.resample(frecuencia).min().dropna()
  df['Close']  = intrad.Close.resample(frecuencia).last().dropna()
  df['Volume'] = intrad.Volume.resample(frecuencia).sum().dropna()
  df['WAP']    = (intrad['WAP'] * intrad['Volume']).resample(frecuencia).sum() / intrad.resample(frecuencia)['Volume'].sum().dropna()
  df['Count']  = intrad.Count.resample(frecuencia).sum().dropna()
  return df

In [15]:
# Agrupa por día
diario = agrupar_temporalmente(intrad)

In [16]:
diario.tail()

,Open,High,Low,Close,Volume,WAP,Count
fecha,,,,,,,
2020-02-10,314.18,321.54,313.85,321.54,390326,318.913508,192070
2020-02-11,323.43,323.90,318.71,319.61,176730,321.534706,89987
2020-02-12,321.55,327.21,321.47,327.20,195834,324.451428,98276
2020-02-13,324.25,326.22,323.35,324.91,173043,325.066101,86295
2020-02-14,324.71,325.98,322.85,324.95,144690,324.574781,70600


In [17]:
diario.head()

,Open,High,Low,Close,Volume,WAP,Count
fecha,,,,,,,
2009-12-22,28.49,28.69,28.38,28.64,825776,28.546081,63521
2009-12-23,28.74,28.91,28.69,28.88,818825,28.825380,60447
2009-12-24,29.08,29.91,29.04,29.86,1195929,29.649044,86052
2009-12-28,30.25,30.56,29.94,30.23,1510775,30.299880,104496
2009-12-29,30.38,30.39,29.82,29.87,1032129,30.027941,73276


In [20]:
# Graficar
fig = go.Figure(data=[go.Candlestick(x=diario.index,
                               open=diario.Open, 
                               high=diario.High,
                               low=diario.Low, 
                               close=diario.Close)])
fig.update_layout(autosize=True)

In [23]:
fig2 = go.Figure(data=[go.Candlestick(x=diario.index,
                               open=diario.Open, 
                               high=diario.High,
                               low=diario.Low, 
                               close=diario.Close)])
fig2.update_layout(
    xaxis_rangeslider_visible=False,
    title='Gráfico Diario',
    yaxis_title='AAPL Stock',
  )

fig2.show()


# Feature Engineering

Ver tf.keras.utils.normalize

In [24]:
# Creamos una función para normalizar y armar train y test por ventanas de tiempo

def multivariate_data(dataset, target_col, train_perc, history_size, future_target, step = 1):
  
  train_split = int(round(dataset.shape[0] * train_perc ,0))
  
  scaler = MinMaxScaler(feature_range=(0,1))
  dataset = scaler.fit_transform(dataset)

  # Crear listas vacias y separar el Target
  x_train, y_train, x_valid, y_valid  = [], [], [], []
  target = dataset[:, target_col]

  # Armar las observaciones de Train
  for i in range(history_size, train_split):
    indices = range(i-history_size, i, step)
    x_train.append(dataset[indices])

    y_train.append(target[i : i + future_target])

  # Armar las observaciones de Valid
  start_index = train_split + history_size
  end_index = len(dataset) - future_target

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    x_valid.append(dataset[indices])

    y_valid.append(target[i : i + future_target])

  return np.array(x_train), np.array(y_train), np.array(x_valid), np.array(y_valid)
  # me devuelve 4 arrays que son x train,  x validacion, y train , y validacion

In [25]:
# Definimos los parámetos para armar la base

train_perc = 0.8    # Porcentaje de obesrvaciones que se usan para Train (el resto es Valid) 
history_size = 10   # Cuantas obesrvaciones pasadas mirar
future_target = 1   # Cuantas observaciones en el futuro tiene que predecir
step = 1            # Pasos que da la ventana por cada observación
target_col = 3      # Columna que se va a usar de target

x_train, y_train, x_val, y_val = multivariate_data(diario, target_col, train_perc, history_size, future_target, step)


In [26]:
y_train

array([[0.00884011],
       [0.00947393],
       [0.0086066 ],
       ...,
       [0.46715815],
       [0.46792541],
       [0.44360677]])

In [27]:
# Armamos el Modelo
model = Sequential()
model.add(LSTM(32, input_shape = x_train.shape[-2:]))
model.add(Dense(1))

# Compilamos el Modelo
model.compile(optimizer= 'adam', loss= 'mean_squared_error')

In [31]:
steps_per_epoch = int( np.ceil(x_train.shape[0] / 8) )
print(steps_per_epoch)
# el 8 es por batch_size

255


In [32]:
# Entrenamos
model.fit(x_train, y_train, batch_size= 8, epochs = 10, 
          steps_per_epoch = 255, validation_data=(x_val, y_val))
# yo cambie el steps_per_epoch porque decia 500 y tiraba un warning

Epoch 1/10
255/255 [==============================] - 2s 6ms/step - loss: 4.8482e-05 - val_loss: 6.7303e-04
Epoch 2/10
255/255 [==============================] - 2s 6ms/step - loss: 5.2359e-05 - val_loss: 3.0932e-04
Epoch 3/10
255/255 [==============================] - 2s 6ms/step - loss: 5.0908e-05 - val_loss: 5.2578e-04
Epoch 4/10
255/255 [==============================] - 2s 6ms/step - loss: 4.1867e-05 - val_loss: 4.1333e-04
Epoch 5/10
255/255 [==============================] - 2s 6ms/step - loss: 4.2652e-05 - val_loss: 2.2086e-04
Epoch 6/10
255/255 [==============================] - 2s 6ms/step - loss: 4.3156e-05 - val_loss: 2.3380e-04
Epoch 7/10
255/255 [==============================] - 2s 6ms/step - loss: 4.0894e-05 - val_loss: 2.8172e-04
Epoch 8/10
255/255 [==============================] - 2s 6ms/step - loss: 4.1916e-05 - val_loss: 3.1216e-04
Epoch 9/10
255/255 [==============================] - 2s 6ms/step - loss: 4.0258e-05 - val_loss: 1.8932e-04
Epoch 10/10
255/255 [=======

In [36]:
y_train

array([[0.00884011],
       [0.00947393],
       [0.0086066 ],
       ...,
       [0.46715815],
       [0.46792541],
       [0.44360677]])



---



# Notas

In [ ]:
# Opciones de resample
# B         business day frequency
# C         custom business day frequency (experimental)
# D         calendar day frequency
# W         weekly frequency
# M         month end frequency
# SM        semi-month end frequency (15th and end of month)
# BM        business month end frequency
# CBM       custom business month end frequency
# MS        month start frequency
# SMS       semi-month start frequency (1st and 15th)
# BMS       business month start frequency
# CBMS      custom business month start frequency
# Q         quarter end frequency
# BQ        business quarter endfrequency
# QS        quarter start frequency
# BQS       business quarter start frequency
# A         year end frequency
# BA, BY    business year end frequency
# AS, YS    year start frequency
# BAS, BYS  business year start frequency
# BH        business hour frequency
# H         hourly frequency
# T, min    minutely frequency
# S         secondly frequency
# L, ms     milliseconds
# U, us     microseconds
# N         nanoseconds